In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,TimeDistributed,SimpleRNN
import numpy as np
import pandas as pd
import copy
df = pd.read_csv("../Data/data_8_nodes_yen.csv") #You need to change #directory accordingly
df.head()

,"X(1, 2)","X(1, 3)","X(2, 4)","X(2, 5)","X(3, 4)","X(3, 7)","X(4, 6)","X(5, 6)","X(5, 8)","X(6, 7)",...,"Y4(1, 3)","Y4(2, 4)","Y4(2, 5)","Y4(3, 4)","Y4(3, 7)","Y4(4, 6)","Y4(5, 6)","Y4(5, 8)","Y4(6, 7)","Y4(7, 8)"
0,10,40,30,50,30,20,20,10,90,70,...,0,1,0,0,0,1,0,0,1,1
1,90,50,50,30,30,80,80,30,30,80,...,1,0,0,1,0,1,1,1,0,0
2,10,60,80,50,40,50,30,90,80,30,...,1,0,0,1,0,1,0,0,1,1
3,80,20,40,60,50,80,90,60,80,80,...,1,0,0,1,0,1,1,1,0,0
4,20,70,50,40,20,70,80,80,10,10,...,0,1,0,0,0,1,1,1,0,0


In [2]:
edges = [[1, 2], [1, 3], [2, 4], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 7], [7, 8]]

In [3]:
number_edges = 11
number_steps = 4
start_node = 1
end_node = 8

In [4]:
X = df.iloc[:,:number_edges].values
y = df.iloc[:,number_edges:].values

In [5]:
X.shape

(100000, 11)

In [6]:
y.shape

(100000, 44)

In [7]:
from sklearn.model_selection import train_test_split
X1,X2,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [8]:
X2[1]

array([30, 80, 20, 80, 80, 70, 60, 30, 40, 20, 50], dtype=int64)

In [9]:
y_test[1]

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
      dtype=int64)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X1)
X_test= sc.fit_transform(X2)

In [11]:
X_train = np.repeat(X_train[:, np.newaxis, : ], number_steps, axis=1)
X_train.shape

(90000, 4, 11)

In [12]:
X_test = np.repeat(X_test[:, np.newaxis, : ], number_steps, axis=1)
X_test.shape

(10000, 4, 11)

In [13]:
y_train[0]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1],
      dtype=int64)

In [14]:
y_train_2 = np.reshape(y_train,(y_train.shape[0], number_steps, number_edges))
y_train_2.shape

(90000, 4, 11)

In [15]:
y_train_2[0]

array([[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1]], dtype=int64)

In [16]:
y_test_2 = np.reshape(y_test,(y_test.shape[0], number_steps, number_edges))
y_test_2.shape

(10000, 4, 11)

In [17]:
X2[1,:]

array([30, 80, 20, 80, 80, 70, 60, 30, 40, 20, 50], dtype=int64)

In [18]:
class Network:
    def __init__(self):
        self.model = Sequential()
        self.model.add(SimpleRNN(16, input_shape=(number_steps, number_edges), 
                            return_sequences=True))
#         model.add(SimpleRNN(64, return_sequences=True))
        self.model.add(TimeDistributed(Dense(number_edges, activation='sigmoid')))
        self.model.compile(loss='BinaryCrossentropy', optimizer='adam')
        self.fitness = float('inf')

    def train(self):
        self.model.fit(X_train, y_train_2, epochs=100, batch_size=64)
    
    def evaluate(self):
        loss = self.model.evaluate(X_train, y_train_2)
        return loss
    
    def set_weights(self, weights):
        self.model.set_weights(weights) 
    
    def get_weights(self):
        return self.model.get_weights()
    
    def architecture(self):
        self.model.summary()

In [19]:
class GeneticAlgorithm:
    def __init__(self, N, Max, Pc, Pm, Ts):
        self.N = N
        self.Max = Max
        self.Pm = Pm
        self.Pc = Pc
        self.population = [Network() for i in range(self.N)]
        self.best = Network()
        self.Ts = Ts
        self.children_population_weights = []
    
    def Train_generation(self):
        i =  0
        for member in self.population:
            print("Train for member: ",i)
            i+=1
            member.train()
            
    def Train_best(self):
        self.best.train()
        
    def Calculate_fitness(self):
        for i in range(len(self.population)):
            self.population[i].fitness = self.population[i].evaluate()
        
#    trao doi cheo 1 thanh phan weights
    def Crossover(self):
        self.children_population_weights.clear()
        for i in range(0,self.N,2):
            father = np.random.randint(0,self.N)
            mother = np.random.randint(0,self.N)
            father_weights = self.population[father].get_weights()
            mother_weights = self.population[mother].get_weights()
            if np.random.uniform(0,1) < self.Pc:           
                mid_point = np.random.choice([i for i in range(len(father_weights))])
                temp = copy.deepcopy(father_weights[mid_point])
                father_weights[mid_point] = copy.deepcopy(mother_weights[mid_point])
                mother_weights[mid_point] = copy.deepcopy(temp)
                self.children_population_weights.append(father_weights)
                self.children_population_weights.append(mother_weights)
            else:
                self.children_population_weights.append(father_weights)
                self.children_population_weights.append(mother_weights)
        for i in range(len(self.children_population_weights)):
            self.population = [Network() for i in range(len(self.children_population_weights))]
            self.population[i].set_weights(self.children_population_weights[i])
        
#     mutation 1 thanh phan weight
    def Mutation(self):
        for member in self.population:
            if np.random.uniform(0,1) < self.Pm:
                old_weight = member.get_weights()
                i = np.random.randint(0,len(old_weight))
                sigma = old_weight[i].std()
                addition = np.random.normal(0, sigma, size=old_weight[i].shape)
                old_weight[i] = old_weight[i] + addition
                member.set_weights(old_weight)
    
    def Selection(self):
        self.children_population_weights.clear()
        t = []
        for i in range(self.N):
            selected_index = np.random.randint(0,len(self.population))
            for ix in np.random.randint(0,len(self.population),self.Ts-1):
                if(self.population[ix].fitness < self.population[selected_index].fitness):
                    selected_index = ix
            t.append(self.population[selected_index].fitness)
            self.children_population_weights.append(self.population[selected_index].get_weights())
        for i in range(self.N):
            self.population[i].set_weights(self.children_population_weights[i])
            self.population[i].fitness = t[i]
    
    def GetBest(self):
        self.population.sort(key=lambda x: x.fitness)
        if self.population[0].fitness < self.best.fitness:
            self.best.set_weights(self.population[0].get_weights())
            self.best.fitness = self.population[0].fitness

    def Do(self):
        for i in range(self.Max):
            self.Crossover()
            self.Mutation()
            self.Calculate_fitness()
            self.Selection()
            self.GetBest()
            print("best fitness:",self.best.fitness)
            print("i = ",i)
        self.Train_best()

In [20]:
N = 5
Max = 10
Pc = 0.5
Pm = 0.05
Ts = 2

In [21]:
GA = GeneticAlgorithm(N, Max, Pc, Pm, Ts)
GA.Do()

2813/2813 [==============================] - 5s 2ms/step - loss: 0.7502
best fitness: 0.7165676355361938
i =  0
2813/2813 [==============================] - 5s 2ms/step - loss: 0.7395
best fitness: 0.7165676355361938
i =  1
2813/2813 [==============================] - 5s 2ms/step - loss: 0.7395
best fitness: 0.7165676355361938
i =  2
2813/2813 [==============================] - 5s 2ms/step - loss: 0.7395
best fitness: 0.7165676355361938
i =  3
2813/2813 [==============================] - 5s 2ms/step - loss: 0.7395
best fitness: 0.7165676355361938
i =  4
2813/2813 [==============================] - 5s 2ms/step - loss: 0.7351
best fitness: 0.7165676355361938
i =  5
2813/2813 [==============================] - 5s 2ms/step - loss: 0.7492
best fitness: 0.7165676355361938
i =  6
2813/2813 [==============================] - 5s 2ms/step - loss: 0.7370
best fitness: 0.7165676355361938
i =  7
2813/2813 [==============================] - 5s 2ms/step - loss: 0.7322
best fitness: 0.7165676355361938

Epoch 42/100
1407/1407 [==============================] - 3s 2ms/step - loss: 0.3030
Epoch 43/100
1407/1407 [==============================] - 3s 2ms/step - loss: 0.3023
Epoch 44/100
1407/1407 [==============================] - 3s 2ms/step - loss: 0.3015
Epoch 45/100
1407/1407 [==============================] - 3s 2ms/step - loss: 0.3010
Epoch 46/100
1407/1407 [==============================] - 3s 2ms/step - loss: 0.3003
Epoch 47/100
1407/1407 [==============================] - 4s 3ms/step - loss: 0.2996
Epoch 48/100
1407/1407 [==============================] - 3s 2ms/step - loss: 0.2990
Epoch 49/100
1407/1407 [==============================] - 3s 2ms/step - loss: 0.2985
Epoch 50/100
1407/1407 [==============================] - 3s 2ms/step - loss: 0.2980
Epoch 51/100
1407/1407 [==============================] - 3s 2ms/step - loss: 0.2974
Epoch 52/100
1407/1407 [==============================] - 4s 2ms/step - loss: 0.2969
Epoch 53/100
1407/1407 [==============================] - 3s 2ms/